In [3]:
from skimage.transform import AffineTransform, SimilarityTransform
# import math
import numpy as np
# from helper import DataLoader, Transform
# import matplotlib.pyplot as plt
# from skimage import data
# from skimage import transform as tf
# import json
# import cv2
from itertools import groupby
import glob
import pandas as pd
# from math import ceil
import os
import scipy.io

In [2]:
def affine_trans(kp, data_name=None, matrix=None):
    src = np.float32([kp[0], kp[16], kp[27], kp[33], kp[39], kp[42]])
    dst = np.float32([[300,900], [1100, 900], [700, 900], [700, 1300],[600,1000],[800,1000]])
    
    if data_name=="bp4d":
        src = np.float32([kp[27], kp[33], kp[39], kp[42], kp[36], kp[45]])
        dst = np.float32([[700, 900], [700, 1300],[600,1000],[800,1000], [400, 1000], [1000, 1000]])

    src_h = np.concatenate((src.T, np.ones(src.shape[0]).reshape(1,src.shape[0])), axis=0)
    dst_h = np.concatenate((dst.T, np.ones(dst.shape[0]).reshape(1,dst.shape[0])), axis=0)
    
    kp_h = np.concatenate((kp.T, np.ones(kp.shape[0]).reshape(1,kp.shape[0])), axis=0)
    
    if matrix is None: matrix = dst_h@np.linalg.pinv(src_h)
    kp_h = matrix@kp_h
    
    return matrix, kp_h[:2, :].T

In [5]:
# just for demo

src = np.float32([[2,3], [5,9]])
dst = np.float32([[400,1000], [600,1000]])
tform = SimilarityTransform()
tform.estimate(src, dst)
print(tform)

src = np.float32([[2,3]])
dst = np.float32([[600,1000]])
tform = SimilarityTransform()
tform.estimate(src, dst)
print(tform)



<SimilarityTransform(matrix=
    [[ 1.33333328e+01,  2.66666656e+01,  2.93333342e+02],
     [-2.66666656e+01,  1.33333328e+01,  1.01333333e+03],
     [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])>
<SimilarityTransform(matrix=
    [[nan, nan, nan],
     [nan, nan, nan],
     [nan, nan, nan]])>


In [3]:
def similarity_trans(kp, matrix=None): 
    
    # Registering right eyebrow and eyes
    src = kp[[36,39], :]
    dst = np.float32([[400, 1000], [600, 1000]])
    tform = SimilarityTransform()
    tform.estimate(src, dst)
    kp[17:22, :] = tform(kp[17:22, :])
    kp[36:42, :] = tform(kp[36:42, :])  
    
    # Registering left eyebrow and eyes
    src = kp[[42,45], :]
    dst = np.float32([[800, 1000], [1000, 1000]])
    tform = SimilarityTransform()
    tform.estimate(src, dst)
    kp[22:27, :] = tform(kp[22:27, :])
    kp[42:48, :] = tform(kp[42:48, :]) 

    # Registering jawline
    src = kp[[0,16], :]
    dst = np.float32([[300, 1000], [1100, 1000]])
    tform = SimilarityTransform()
    tform.estimate(src, dst)
    kp[0:17, :] = tform(kp[0:17, :])
    
    # Registering nose
    delta_x = 700 - kp[27, 0] 
    delta_y = 1000 - kp[27, 1]
    kp[27:36, 0], kp[27:36, 1] =  kp[27:36, 0]+delta_x, kp[27:36, 1]+delta_y
    
    return np.array(matrix), kp

In [4]:
n_kp=68 # For any data, since all are vectorized to 68 keypoints

In [5]:
columns = ['Dataset', 'Subject_id', 'Video_id', 'Offset_Frame', 'Apex_Frame']

for i in range(n_kp):
  columns.append("Offset_x"+str(i))
  columns.append("Offset_y"+str(i))
    
for i in range(n_kp):
  columns.append("Apex_x"+str(i))
  columns.append("Apex_y"+str(i))
    
for i in range(n_kp):
  columns.append("Offset_x"+str(i)+"_Normalized")
  columns.append("Offset_y"+str(i)+"_Normalized")
    
for i in range(n_kp):
  columns.append("Apex_x"+str(i)+"_Normalized")
  columns.append("Apex_y"+str(i)+"_Normalized")

for i in range(9):
    columns.append("offset_a"+str(i))
    
for i in range(9):
    columns.append("apex_a"+str(i))

for i in range(100):
  columns.append("AU"+str(i+1))

In [6]:
len(pd.DataFrame(columns = columns).columns) 

667

# DISFA keypoint dataset creation

In [ ]:
from FacialLandmarkFrontalization_66.source.utils import frontalize_landmarks
wts = np.load("FacialLandmarkFrontalization_66/data/frontalization_weights.npy")

In [ ]:
gt_output_path =  "../../datasets/macro/Spontaneous/DISFA/DISFA/Landmark_Points/"
au_coding_path = "../../datasets/macro/Spontaneous/DISFA/DISFA/ActionUnit_Labels/"

In [ ]:
frame_path_list = glob.glob("/mnt/data/StrokeML/Macroexpression/DISFA/DISFA/Frames_LeftCamera/*/", recursive=True)
kp_path_list = glob.glob(au_coding_path+"../Landmark_Points/*/", recursive=True)
frame_path_list.sort()
kp_path_list.sort()
count = 0

for frame_path, kp_path in zip(frame_path_list, kp_path_list):
#     print(frame_path)
    jpg_list = glob.glob(frame_path+"**/*.jpg", recursive=True)
    kp_list = glob.glob(kp_path+"**/*.mat", recursive=True)

    jpg_list.sort()
    kp_list.sort()
    if len(jpg_list)!=len(kp_list):
        print(frame_path)

In [ ]:
ignore_sub_left = [4, 16]

In [ ]:
global_neutral_frame = 3741

### Creating pair list for all frames for the data

In [ ]:
landmark_pt_path = "../../datasets/macro/Spontaneous/DISFA/DISFA/Landmark_Points/"

In [ ]:
pair_list_all_frames = []
sub_no_list = []
for sub in glob.glob(landmark_pt_path+"*/*/", recursive=True):
    sub_split = sub.split("/")
    sub_no = int(sub_split[-3][3:5])
    sub_no_list.append(sub_no)
    print("Subject No : ", sub_no)
    offset = global_neutral_frame 

    for apex in range(len(os.listdir(sub))):
        if apex!=global_neutral_frame:
            pair = [0, sub_no, offset, apex, 0]
            pair_list_all_frames.append(pair)

print("Total frames : ", len(pair_list_all_frames))

In [ ]:
sub_no_list = list(pd.DataFrame(sub_no_list)[0].unique())

In [ ]:
# sub_no_list

In [ ]:
sub_no_left = list(set(sub_no_list) - set(ignore_sub_left))

In [ ]:
kp_l = []

for sub in sub_no_left:
    mat_path = gt_output_path+"SN0"+f"{sub:02}"+"/"
#     print(mat_path)
    kp_list = glob.glob(mat_path+"*/*.mat", recursive=True)
    kp_list.sort()
    
    video = np.zeros((len(kp_list), 66, 2))
    for i in range(len(kp_list)):
        mat = scipy.io.loadmat(kp_list[i])
        video[i, :, :] = mat['pts']
    
    kp_l.append(video) 

In [ ]:
def disfa_66_to_68(kp):
    kp_new = np.zeros((68, 2))
    kp_new[:60, :] = kp[:60, :]
    kp_new[61:64, :] = kp[60:63, :]
    kp_new[65:68, :] = kp[63:66, :]
    return kp_new

In [ ]:
def create_final_data(pair_list):
# 
    au_list = list([1,2,4,5,6,9,12,15,17,20,25,26])

    index_list = [0, 16, 27, 33, 39, 42, 36, 45]
    X_ori = np.float32([[300,900], [1100, 900], [700, 915], [700, 1300],[600,920],[800,920], [400, 910], [1000, 910]])


    final_data = []

    for key in pair_list:

        try:
            res2 = []
            au_array = np.zeros(100)
            res2.extend(('DISFA', "LeftVideoSN0"+f"{key[1]:02}", 'Videos_LeftCamera'))
            video_l = kp_l[sub_no_left.index(key[1])]
        
            offset = video_l[key[2], :, :]
            offset = frontalize_landmarks(offset, wts)
            offset = 200*(offset+np.amax(offset))
            offset_matrix, offset_n = affine_trans(offset)
            _, offset_n = similarity_trans(offset_n)   
            offset = disfa_66_to_68(offset)
            offset_n = disfa_66_to_68(offset_n)

            apex = video_l[key[3], :, :]
            apex = frontalize_landmarks(apex, wts)
            apex = 200*(apex+np.amax(apex))
            apex_matrix, apex_n = affine_trans(apex)
            _, apex_n = similarity_trans(apex_n)
            apex = disfa_66_to_68(apex)
            apex_n = disfa_66_to_68(apex_n)      

            res2.extend(np.concatenate(([key[2]], [key[3]], offset.flatten(), apex.flatten(), offset_n.flatten(), apex_n.flatten(), offset_matrix.flatten(), apex_matrix.flatten(), au_array.flatten())))
            final_data.append(res2)

        except Exception as e:
            print(e)

    return final_data

In [ ]:
final_data_all_frames = create_final_data(pair_list_all_frames)

In [ ]:
# df_disfa = pd.DataFrame(final_data, columns=columns)
# cols_groupby = columns[:-100]
# cols_au = columns[-100:]
# df_temp = df_disfa.copy()
# df_temp = df_temp.groupby(cols_groupby)[cols_au].sum().reset_index()
# df_temp.to_csv("Data/reg1_6_pts/cum_event_front_facing_2level_reg_eyes_eyebrows_jawline_nose/left_DISFA_GlobalNeutral.csv")

In [ ]:
# len(final_data_all_frames)

In [ ]:
len(final_data_all_frames) #121099

In [ ]:
df_disfa = pd.DataFrame(final_data_all_frames, columns=columns)
cols_groupby = columns[:-100]
cols_au = columns[-100:]
df_temp = df_disfa.copy()
df_temp = df_temp.groupby(cols_groupby)[cols_au].sum().reset_index()
# df_temp.to_csv("Data/reg1_6_pts/cum_event_front_facing_2level_reg_eyes_eyebrows_jawline_nose/left_DISFA_GlobalNeutral_all_frames_8p.csv")

In [ ]:
df_temp.shape

### Eliminating frames from Error_LOG_Sheet

In [ ]:
# df_temp = pd.read_csv("Data/reg1_6_pts/cum_event_front_facing_2level_reg_eyes_eyebrows_jawline_nose/left_DISFA_GlobalNeutral_all_frames_8p.csv", index_col=0)

In [ ]:
error_sheet_path = "../../datasets/macro/Spontaneous/DISFA/DISFA/Error_LOG_Sheet.xls"

In [ ]:
df_error = pd.read_excel(error_sheet_path, sheet_name="Sheet1", ).iloc[5:19, :]

In [ ]:
df_error

In [ ]:
# df_temp_copy = df_temp.copy()

In [ ]:
# df_temp_copy.shape 

In [ ]:
# df_temp.shape

In [ ]:
# df_temp = df_temp_copy.copy()

In [ ]:
for i in range(df_error.shape[0]-1):
    df_error_sub = df_error.iloc[i, :].dropna()
    print(df_error_sub)
    if df_error_sub[0]=="SN011":
        df_error_sub = df_error_sub[:-1]
    subject_name = "LeftVideo"+df_error_sub[0]
    print(subject_name)
    error_frames = np.array(df_error_sub[1:]).reshape(-1, 2)
    for start_error_frame, end_error_frame in zip(error_frames[:, 0], error_frames[:, 1]):        
        df_temp.drop(df_temp[(df_temp["Subject_id"]==subject_name) & (df_temp["Offset_Frame"]>=start_error_frame) & (df_temp["Offset_Frame"]<=end_error_frame)].index, inplace=True)
        df_temp.drop(df_temp[(df_temp["Subject_id"]==subject_name) & (df_temp["Apex_Frame"]>=start_error_frame) & (df_temp["Apex_Frame"]<=end_error_frame)].index, inplace=True)

In [ ]:
df_temp.to_csv("Data/DISFA_full.csv")

In [ ]:
print(df_temp.shape) #114654

In [ ]:
df_temp = df_temp.sample(frac=1, random_state=17).iloc[:10100, :]
df_temp.reset_index(inplace=True)
df_temp.drop(columns=['index'], inplace=True)

In [ ]:
df_temp.columns

In [ ]:
df_temp.to_csv("Data/DISFA_train.csv")

# BP4D-Spontaneous keypoint dataset creation

### Creating csv for intensity coded files

In [ ]:
au_coding_path = "../../datasets/macro/Spontaneous/BP4D_Spontaneous/AUCoding/AU_INT/"
au_coding_new_path = "../../datasets/macro/Spontaneous/BP4D_Spontaneous/AUCoding_new/AU_INT/"
# alpha_path = "../../project/Face_Tracking/alphapose/outputs/BP4D_Spontaneous/"
img_path = "/mnt/scratch2/csy207576/project/datasets/macro/Spontaneous/BP4D_Spontaneous/Sequences(2D+3D)/"

In [ ]:
count = 0
csv = 0
for csv_path in glob.glob(au_coding_path+"*/*.csv", recursive=True):
    csv += 1
    
    au_coding = pd.read_csv(csv_path, header=None)
#     print(au_coding)
    length = au_coding[0][au_coding.shape[0]-1] - au_coding[0][0] + 1
#     print(csv_path, length)
    if length==au_coding.shape[0]:
        count+=1
print(count, csv)

# The above shows that the frame numbers are consistent without any gaps from start to end in csv files

In [ ]:
# ignore_json_list = []
# ori_count = 0
# check_count = 0

# for txt_path in glob.glob(au_coding_path+"*/*.csv", recursive=True):
#     ori_count += 1
#     sub_id = txt_path.split("/")[-1][:4]
#     video_id = txt_path.split("/")[-1][5:7]
    
#     img = img_path+sub_id+"/"+sub_id+"/"+video_id+"/*.jpg"
#     img_length = len(glob.glob(img_path+sub_id+"/"+sub_id+"/"+video_id+"/*.jpg", recursive=True))

#     json_path = alpha_path + sub_id + "/" + video_id + "/" + "alphapose-results.json"
    
#     json_length = len(json.load(open(json_path)))

#     if img_length==json_length:
#         check_count += 1
#     else:
#         ignore_json_list.append(txt_path)

In [ ]:
# print(ori_count, check_count)

In [ ]:
# np.arange(1, img_length+1, 1)

In [ ]:
# # Creating new AU Files that are similar to DISFA+ coding

# for csv_path in glob.glob(au_coding_path+"*/*.csv", recursive=True):
#     print(csv_path)
#     sub_id = csv_path.split("/")[-1][:4]
#     video_id = csv_path.split("/")[-1][5:7]
    
#     temp_img_path = img_path+sub_id+"/"+sub_id+"/"+video_id+"/*.jpg"
#     img_length = len(glob.glob(temp_img_path, recursive=True))

#     au_new = np.zeros((img_length, 2))
#     au_coding = pd.read_csv(csv_path, header=None)
    
#     start = au_coding.iloc[0, 0]
#     end = start + au_coding.shape[0] - 1

#     au_new[:, 0] = np.arange(1, img_length+1, 1)

#     au_new[(start-1):end, 1] = np.array(au_coding.iloc[:, 1])
    
#     new_au_path = au_coding_new_path +csv_path.split("/")[-2]+"/"+csv_path.split("/")[-1][:-4]+".txt"
    
#     au_new = np.array(au_new, dtype=np.int64)
#     np.savetxt(new_au_path, au_new, delimiter=' ', newline='\n')



In [ ]:
sub_list = []
for i in range(1, 24):
    sub_list.append("F"+str(i).zfill(3))
for i in range(1, 19):
    sub_list.append("M"+str(i).zfill(3))
    
print(sub_list)

In [ ]:
from FacialLandmarkFrontalization_49.source.utils import frontalize_landmarks
wts = np.load("FacialLandmarkFrontalization_49/data/frontalization_weights.npy")


def kp_preprocess_bp4d(kp):
    kp_temp = np.zeros((66,2))
    kp_temp[17:, :] = kp
    kp_temp = disfa_66_to_68(kp_temp)
    kp_temp[:17, :] = np.zeros((17, 2))
    return kp_temp


def kp_preprocess_bp4d_n(kp):
    kp_temp = kp.copy()
    kp_temp[:, 0] = kp_temp[:, 0] - np.amin(kp_temp[:, 0]) + 50
    kp_temp[:, 1] = kp_temp[:, 1] - np.amin(kp_temp[:, 1]) + 50

    kp_temp = kp_preprocess_bp4d(kp_temp)
    kp_temp = frontalize_landmarks(kp_temp, wts)
    kp_temp += np.amax(kp_temp)
    kp_temp = 200*kp_temp
    matrix, kp_temp = affine_trans(kp_temp, "bp4d")
    _, kp_temp = similarity_trans(kp_temp)
    kp_temp[:17, :] = np.zeros((17, 2))
    kp_temp[60, :] = [0,0]
    kp_temp[64, :] = [0,0]
    return matrix, kp_temp

In [ ]:
# Listing the Offset frames per subject
import glob
import itertools
au_csv_path = "../../datasets/macro/Spontaneous/BP4D_Spontaneous/AUCoding/AU_OCC/"
kp_path = "../../datasets/macro/Spontaneous/BP4D_Spontaneous/2DFeatures_temp/"
global_min_list = []
pair_list = []
final_data = []
count = 0

for sub in sub_list:
    count+=1
    apex_list = []
    min_list = ["", "", -10, -10000, 10000] # sub, vid, min_index, freq
    for au_csv in glob.glob(au_csv_path+sub+"*.csv", recursive=True):
        
        vid_id = au_csv[-6:-4]
        df_csv = pd.read_csv(au_csv)
        df_csv.replace(9, 0, inplace=True)
        df_sum = df_csv.iloc[:, 1:].sum(axis=1)
        
        
        for t in range(df_sum.shape[0]):
            if df_sum.iloc[t]>0:
                apex_frame = df_csv.iloc[t, 0]
                try:
                    apex = np.loadtxt(kp_path+sub+"/"+sub+"/"+vid_id+"/"+str(apex_frame).zfill(4)+".txt", delimiter=",")
                    apex_list.append([apex_frame, apex, sub, vid_id])
                except:
                    apex = np.loadtxt(kp_path+sub+"/"+sub+"/"+vid_id+"/"+str(apex_frame).zfill(3)+".txt", delimiter=",")
                    apex_list.append([apex_frame, apex, sub, vid_id])
        
        
        df_sum = df_sum.to_numpy()
        arr = np.zeros(df_sum.shape[0])
        indices = np.where(df_sum==np.amin(df_sum))
        arr[indices] = 1
        freq = [(k, len(list(j))) for k, j in groupby(list(arr))]
        max_ind = -10
        for k in range(len(freq)):
            if freq[k][0] == 0:
                continue
            else:
                if max_ind == -10:
                    max_ind = k
                elif freq[k][1]> freq[max_ind][1]:
                    max_ind = k
        freq = np.array(freq)
        max_index_csv = int(np.sum(freq[:max_ind, 1]) + np.sum(freq[max_ind, 1])//2)
        
        if min_list[4]>np.amin(df_sum):
            min_list = [sub, vid_id, df_csv.iloc[max_index_csv,0], freq[max_ind, 1], np.amin(df_sum)]
        elif min_list[4]==np.amin(df_sum):
            if min_list[3]<freq[max_ind, 1]:
                min_list = [sub, vid_id, df_csv.iloc[max_index_csv,0], freq[max_ind, 1], np.amin(df_sum)]                
        
    print(min_list)
        
    sub = min_list[0]
    vid_id = min_list[1]
    offset_vid = vid_id
    offset_frame = min_list[2]
    try:
        offset = np.loadtxt(kp_path+sub+"/"+sub+"/"+vid_id+"/"+str(offset_frame).zfill(4)+".txt", delimiter=",")
    except:
        offset = np.loadtxt(kp_path+sub+"/"+sub+"/"+vid_id+"/"+str(offset_frame).zfill(3)+".txt", delimiter=",")
    
    
    offset_matrix, offset_n = kp_preprocess_bp4d_n(offset)
    # the above code also changed values in offset since a reference is passed and not a copy
    offset = kp_preprocess_bp4d(offset)
    for apex_pair in apex_list:
        
        try:
            apex = apex_pair[1]
            sub = apex_pair[2]
            vid_id = apex_pair[3]
            apex_matrix, apex_n = kp_preprocess_bp4d_n(apex)
            # the above code also changed values in offset since a reference is passed and not a copy
            apex_frame = apex_pair[0]

            apex = kp_preprocess_bp4d(apex)
            res = []   
            res.extend(["BP4D_Spontaneous", sub, offset_vid+"_"+vid_id, offset_frame, apex_frame])
            res.extend(np.concatenate((offset.flatten(), apex.flatten(), offset_n.flatten(), apex_n.flatten(), offset_matrix.flatten(), apex_matrix.flatten(), np.zeros((100)))))
            final_data.append(res)
#         rows_count+=1
#         if rows_count>=10:
#             break
        except Exception as e:
            print(str(e))
#     if rows_count>=10:
#         break
        
    global_min_list.append(min_list)
        

In [ ]:
df_bp4d = pd.DataFrame(final_data, columns=columns)
cols_groupby = columns[:-100]
cols_au = columns[-100:]
df_temp = df_bp4d.copy()
df_temp = df_temp.groupby(cols_groupby)[cols_au].sum().reset_index()
df_temp.to_csv("Data/BP4D_full.csv")

In [ ]:
df_sample = df_temp.sample(frac=1, random_state=17).iloc[:10100, :]
df_sample.reset_index(inplace=True)
df_sample.drop(columns=['index'], inplace=True)
df_sample.to_csv("Data/BP4D_train.csv")

In [ ]:
df_temp.shape

# CK+ keypoint dataset creation

In [74]:
from FacialLandmarkFrontalization.source.utils import frontalize_landmarks
wts = np.load("FacialLandmarkFrontalization/data/frontalization_weights.npy")

In [75]:
gt_path = "../../datasets/macro/Both/CK+/CK+/Landmarks/"
au_path = "../../datasets/macro/Both/CK+/CK+/FACS/"

final_data_all_frames = []

check_count = 0

for kp_file_path in glob.glob(gt_path+"*/*/", recursive=True):
    
    video = os.listdir(kp_file_path)
    video.sort()

    sub_id = kp_file_path.split("/")[-3]
    video_id = kp_file_path.split("/")[-2]
    
    au = np.loadtxt(open(glob.glob(au_path + sub_id +"/" + video_id + "/*.txt", recursive=True)[0]))
    
    au_array = np.zeros(100)
    
    try:
        temp = au.shape[0]*au.shape[1]
        for row in au:
            au_array[int(row[0]) - 1] = row[1]
    except Exception as e:
        print(str(e))
        au_array[int(au[0]) - 1] = au[1]


    offset_frame = 0
    for apex_frame in range(1, len(video)):
        
        try:
            offset = np.loadtxt(kp_file_path+video[0])
            offset_n = frontalize_landmarks(offset, wts)
    #         offset = 200*(offset+np.amax(offset))
            offset_matrix, offset_n = affine_trans(offset_n)
            _, offset_n = similarity_trans(offset_n)

#             apex = np.loadtxt(kp_file_path+video[-1])
            apex = np.loadtxt(kp_file_path+video[apex_frame])
            apex_n = frontalize_landmarks(apex, wts)
    #         apex = 200*(apex+np.amax(apex))
            apex_matrix, apex_n = affine_trans(apex_n)
            _, apex_n = similarity_trans(apex_n)
            res = []
            res.extend(['CK+', sub_id, video_id, offset_frame, apex_frame])

            res.extend(np.concatenate((offset.flatten(), apex.flatten(), offset_n.flatten(), apex_n.flatten(), offset_matrix.flatten(), apex_matrix.flatten(), au_array.flatten())))
            final_data_all_frames.append(res)
            
        except Exception as e:
            print(str(e))
            pass

tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
tuple index out of range
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
'utf-8' codec can't decode byte

In [76]:
len(final_data_all_frames)

10100

In [77]:
df_CK_plus = pd.DataFrame(final_data_all_frames, columns=columns)

In [78]:
df_CK_plus.to_csv("Data/CK+.csv")

# Pure AUs keypoint dataset creation

In [65]:
from FacialLandmarkFrontalization.source.utils import frontalize_landmarks
wts = np.load("FacialLandmarkFrontalization/data/frontalization_weights.npy")

In [66]:
from pathlib import Path

au_kp_path = "../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/"
apex_list = {'4b':["Brow Lowerer",[17,27],[36,48]] ,'1':["Inner Brow Raiser",[17,27]],'2':["Outer Brow Raiser",[17,27],[36,48]],'5z':["Upper Lid Raiser",[17,27],[36,48]],'7':["Lid Tightener",[17,27],[36,48]],'6':["Cheek Raiser and Lid Compressor",[17,68]],'43':["Eye Closure",[36,48]],'45':["Blink", [36,48]],'46':["Wink",[36,48]],'9':["Nose Wrinkler",[17,68]],'10x':["Upper Lip Raiser",[27,36],[48,68]],'17':["Chin Raiser",[0,17],[48,68]],'15x':["Lip Corner Depressor",[48,68]],'25':["Lips Part",[48,68]],'26':["Jaw Drop",[0,17],[48,68]],'27':["Mouth Stretch",[0,17],[48,68]],'16_25':["Lower Lip Depressor",[0,17],[48,68]],'20x':["Lip Stretcher",[0,17],[48,68]],'14':["Dimpler",[0,17],[48,68]],'11':["Nasolabial Furrow Deepener",[0,68]],'12x':["Lip Corner Puller",[48,68]],'13b':["Sharp Lip Puller",[48,68]],'18a':["Lip Pucker",[48,68]],'22_25':["Lip Funneler",[48,68]],'23':["Lip Tightener",[48,68]],'24':["Lip Presser",[48,68]],'28':["Lips Suck",[48,68]],'21':["Neck Tightener",[0,17]],'31':["Jaw Clencher",[0,17]],'38':["Nostril Dilator",[27,36]],'39':["Nostril Compressor",[27,36]]}

final_data = []

offset = np.loadtxt(au_kp_path+"s0.txt")
offset_n = frontalize_landmarks(offset, wts)
offset_matrix, offset_n = affine_trans(offset_n)
_, offset_n = similarity_trans(offset_n)

for au in apex_list:
    if not Path(au_kp_path+"s"+au+".jpg").exists():
        print(au, "no doesn't exist")
        
    res = []
    au_array = np.zeros(100)
    
    try:
        apex = np.loadtxt(au_kp_path+"s"+au+".txt")
        apex_n = frontalize_landmarks(apex, wts)
        apex_matrix, apex_n = affine_trans(apex_n)
        _, apex_n = similarity_trans(apex_n)
        res.extend(['Pure AU', apex_list[au][0], "NA", "0", au])

        res.extend(np.concatenate((offset.flatten(), apex.flatten(), offset_n.flatten(), apex_n.flatten(), offset_matrix.flatten(), apex_matrix.flatten(), au_array.flatten())))
        final_data.append(res)
    except Exception as e:
        print(str(e))
        pass

45 no doesn't exist
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s45.txt not found.
46 no doesn't exist
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s46.txt not found.
31 no doesn't exist
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s31.txt not found.
38 no doesn't exist
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s38.txt not found.
39 no doesn't exist
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s39.txt not found.


In [67]:
df_AUs = pd.DataFrame(final_data, columns=columns)

In [68]:
df_AUs.to_csv("Data/pureAUs.csv")

# AUs with combination keypoint dataset creation

In [69]:
au_kp_path = "../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/"
apex_list = {'4b':["Brow Lowerer",[17,27],[36,48]] ,'1':["Inner Brow Raiser",[17,27]],'2':["Outer Brow Raiser",[17,27],[36,48]],'5z':["Upper Lid Raiser",[17,27],[36,48]],'7':["Lid Tightener",[17,27],[36,48]],'6':["Cheek Raiser and Lid Compressor",[17,68]],'43':["Eye Closure",[36,48]],'45':["Blink", [36,48]],'46':["Wink",[36,48]],'9':["Nose Wrinkler",[17,68]],'10x':["Upper Lip Raiser",[27,36],[48,68]],'17':["Chin Raiser",[0,17],[48,68]],'15x':["Lip Corner Depressor",[48,68]],'25':["Lips Part",[48,68]],'26':["Jaw Drop",[0,17],[48,68]],'27':["Mouth Stretch",[0,17],[48,68]],'16_25':["Lower Lip Depressor",[0,17],[48,68]],'20x':["Lip Stretcher",[0,17],[48,68]],'14':["Dimpler",[0,17],[48,68]],'11':["Nasolabial Furrow Deepener",[0,68]],'12x':["Lip Corner Puller",[48,68]],'13b':["Sharp Lip Puller",[48,68]],'18a':["Lip Pucker",[48,68]],'22_25':["Lip Funneler",[48,68]],'23':["Lip Tightener",[48,68]],'24':["Lip Presser",[48,68]],'28':["Lips Suck",[48,68]],'21':["Neck Tightener",[0,17]],'31':["Jaw Clencher",[0,17]],'38':["Nostril Dilator",[27,36]],'39':["Nostril Compressor",[27,36]]}
neutral = ['0']

final_data = []

offset = np.loadtxt(au_kp_path+"s0.txt")
offset_n = frontalize_landmarks(offset, wts)
offset_matrix, offset_n = affine_trans(offset_n)
_, offset_n = similarity_trans(offset_n)
print(offset.shape, offset_n.shape)
offset_n_ori = offset_n


for apex_path in glob.glob(au_kp_path+"*.txt"):
    
    temp = apex_path.split("/")[-1]
    if "s0.txt" in temp or "sJ" in temp or "sL" in temp or "sW" in temp: 
        continue
    print(apex_path)
    res = []
    au_array = np.zeros(100)
    
#     try:
    apex = np.loadtxt(apex_path)
    apex_n = frontalize_landmarks(apex, wts)
    apex_matrix, apex_n = affine_trans(apex_n)
    _, apex_n = similarity_trans(apex_n)
    res.extend(['AU_with_combinations', "NA", "NA", "0", temp[:-4]])

    res.extend(np.concatenate((offset.flatten(), apex.flatten(), offset_n.flatten(), apex_n.flatten(), offset_matrix.flatten(), apex_matrix.flatten(), au_array.flatten())))
    final_data.append(res)
#     except Exception as e:
#         print(str(e))
#         pass
    
    
df_AUs = pd.DataFrame(final_data, columns=columns)
print(df_AUs.shape)
df_AUs.to_csv("Data/combAUs.csv")

(68, 2) (68, 2)
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s1.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s1012x25.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s1012y25.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s102x162.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s102y162.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10x.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y1625.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y1723.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y2325.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y_14.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y_15z.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y_17.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10y_20.txt
../../Face_Tracking/SRT/outputs/Ekman/Manual/Examples/s10z1517.

# Creation features for all datasets

In [70]:
d = DataLoader()

# for data_name in ["DISFA_train", "DISFA_full", "BP4D_train", "BP4D_full", "CK+", "pureAUs", "combAUs"]:
for data_name in ["pureAUs","combAUs"]:
    df, _, cols_n = d.delta_loader("Data/"+data_name+".csv")
    X = df[cols_n[0]+cols_n[1]]
    X = X.to_numpy()
    if "pureAUs" in data_name or "combAUs" in data_name:
        X = X/np.linalg.norm(X, axis=1).reshape(X.shape[0], -1)
    pd.DataFrame(X).to_csv("Data/delta_"+data_name+".csv")